# 投票分类器

In [10]:
# 假设你已经训练好了一些分类器，每个分类器的准确率约为80%。
# 大概包括一个逻辑回归分类器、一个SVM分类器、一个随机森林分类器、一个K-近邻分类器，或许还有更多
# 这时，要创建出一个更好的分类器，最简单的办法就是聚合每个分类器的预测，
# 然后将得票最多的结果作为预测类别。这种大多数投票分类器被称为硬投票分类器

In [11]:
# 当预测器尽可能互相独立时，集成方法的效果最优。
# 获得多种分类器的方法之一就是使用不同的算法进行训练。
# 这会增加它们犯不同类型错误的机会，从而提升集成的准确率

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split


X, y = make_moons(n_samples=1000, noise=0.20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(
estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [12]:
# 看一下测试集上每个分类器的精度
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.88
RandomForestClassifier 0.96
SVC 0.97
VotingClassifier 0.97


In [13]:
# 如果所有分类器都能够估算出类别的概率（即有predict_proba（）方法），
# 那么你可以将概率在所有单个分类器上平均，然后让Scikit-Learn给出平均概率最高的类别作为预测。
# 这被称为软投票法。通常来说，它比硬投票法的表现更优，
# 因为它给予那些高度自信的投票更高的权重。
# 而所有你需要做的就是用voting="soft"代替voting="hard"，
# 并确保所有分类器都可以估算出概率。

# bagging和pasting

In [14]:
# 前面提到，获得不同种类分类器的方法之一是使用不同的训练算法。
# 还有另一种方法是每个预测器使用的算法相同，但是在不同的训练集随机子集上进行训练。
# 采样时如果将样本放回，这种方法叫作bagging[1]（bootstrap aggregating[2]的缩写，也叫自举汇聚法）。
# 采样时样本不放回，这种方法则叫作pasting

In [15]:
# Scikit-Learn提供了一个简单的API，可用BaggingClassifier类进行bagging和pasting（或BaggingRegressor用于回归）

# 如果基本分类器可以估计类别概率（如果它具有predict_proba（）方法），
# 则BaggingClassifier自动执行软投票而不是硬投票，在决策树分类器中就是这种情况

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500,           # 500个决策树分类器的集成
                            max_samples=100,            # 每次从训练集中随机采样100个训练实例进行训练
                            bootstrap=True,             # 放回抽样，不放回抽样为False
                            n_jobs=-1)                  # 使用所有cpu内核进行训练

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

                            包外评估                            

In [16]:
# 对于任意给定的预测器，使用bagging，有些实例可能会被采样多次，
# 而有些实例则可能根本不被采样。BaggingClassifier默认采样m个训练实例，
# 然后放回样本（bootstrap=True），m是训练集的大小。
# 这意味着对每个预测器来说，平均只对63%的训练实例进行采样。
# 剩余37%未被采样的训练实例称为包外（oob）实例

# 由于预测器在训练过程中从未看到oob实例，因此可以在这些实例上进行评估，而无须单独的验证集。
# 你可以通过平均每个预测器的oob评估来评估整体

In [19]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),
                            n_estimators=500,
                            bootstrap=True,
                            n_jobs=-1,
                            oob_score=True)         # 请求在训练结束后自动进行包外评估

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.95125

In [20]:
# 根据此oob评估，此BaggingClassifier能在测试集上达到约95%的准确率。让我们验证一下：

from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.96

# 随机补丁和随机子空间

In [ ]:
# BaggingClassifier类也支持对特征进行采样。采样由两个超参数控制：max_features和bootstrap_features。
# 它们的工作方式与max_samples和bootstrap相同，但用于特征采样而不是实例采样。
# 因此，每个预测器将用输入特征的随机子集进行训练。
# 这对于处理高维输入（例如图像）特别有用。
# 对训练实例和特征都进行抽样，这称为随机补丁方法。
# 而保留所有训练实例（即bootstrap=False并且max_samples=1.0）
# 但是对特征进行抽样（即bootstrap_features=True并且/或max_features<1.0），这被称为随机子空间法。
# 对特征抽样给预测器带来更大的多样性，所以以略高一点的偏差换取了更低的方差